<a href="https://colab.research.google.com/github/Zer0Is/Computacion-Cientifica/blob/main/2018735590-tarea-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

##Análisis:
Dado que es un polinomio de grado 3, este debe tener 3 soluciones posibles, por ende se puede factorizar, una de las soluciones se encuentra por el metódo de Ruffini, donde da como solución x = 1, el polinomio queda:

\begin{equation}
f(x)=(x-1)(A^2x^2-2A^2x+1)
\end{equation}

Luego, el polinomio restante es de grado 2, por ende, se aplica la ecuación:

\begin{equation}
\frac{-b \pm \sqrt{b^2-4ac}}{2a}
\end{equation}

Donde 
\begin{equation}
a = A^2
\end{equation}
\begin{equation}
b = -2A^2;
\end{equation}
\begin{equation}
c = 1
\end{equation}        

Las soluciones son:
\begin{equation}
x1 = 1 + \frac{\sqrt{A^2-1}}{A}  
\end{equation}
\begin{equation} 
x2 = 1 - \frac{\sqrt{A^2-1}}{A}
\end{equation} 

Además esta la condición donde A>>1, por ende se puede concluir que:
\begin{equation} 
\sqrt{A^2-1} \approx A
\end{equation}

Por lo tanto:
\begin{equation} 
x1 = 2
\end{equation}
\begin{equation} 
x2 = 0
\end{equation}

La Factorización final queda:
Por lo tanto:
\begin{equation} 
(x-1)(x-2)x
\end{equation}

In [16]:
!pip install bitstring

In [66]:
import bitstring as bs
import numpy as np
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
#Ref[1]
def f_new_rep(x,bits_mant,bits_exp):
  
  #iniciamos resultado en 0
  result = 0

  #Transforma numero en punto flotante con doble precision
  b = bs.pack('>d', x)
  b = b.bin
  mantisa = b[12:]
  print(b[0]+' '+b[1:12]+ ' '+b[12:])

  #Vemos la mantisa
  #Vemos si es necesario truncar o no la mantisa
  if bits_mant < 52:

    #Se revisa el ultimo bit de la "nueva" mantisa
    #es un "0"
    if b[11+bits_mant] == '0':
      
      #Si el primer elemento fuera de la "nueva" mantisa es un "0"
      #simplemente se trunca la mantisa segun "bits_mant"
      if b[12+bits_mant] == '0':
        mantisa = b[12:12+bits_mant]
      
      #Si es un "1", hay que ver si es el caso "1000000", es decir, "justo en la mitad"

      #Si hay algun "1" despues del primer "1", tipo -> "1000001000"
      #se le debe sumar "1" a la mantisa
      elif '1' in b[13+bits_mant:]:
        for bit in range(11+bits_mant,11,-1): #13 a 12
          
          #Se encuentra el "primer cero" desde derecha de la mantisa hacia la izquierda
          if b[bit] == '0':
            #mantienen bits hasta el que hay que cambiar(sin considerarlo)
            #agrega el '1' que reemplaza al '0' encontrado
            #se agrega los '0' restantes para completar la nueva mantisa
            mantisa = b[12:bit] + '1' + '0'*(bits_mant-bit)
            break

      #En caso contrario solo se trunca "hacia abajo"
      else:
        mantisa = b[12:12+bits_mant]
    
    #es un "1"
    else:
      #Si el primer elemento fuera de la "nueva" mantisa es un "1"
      #simplemente se suma un "1" a la "nueva" mantisa
      if b[12+bits_mant] == '1':
        for bit in range(11+bits_mant,11,-1):
          #Se encuentra el "primer cero" desde derecha de la mantisa hacia la izquierda
          if b[bit] == '0':
            mantisa = b[12:bit] + '1' + '0'*(bits_mant-bit)
            break

          #Se evita el OverFlow
          if bit==12:
            mantisa = b[12:12+bits_mant]
      #En caso contrario solo se mantiene la mantisa nueva segun la cantidad de bits que se mencionan con "bits_mant"
      else:
        mantisa = b[12:12+bits_mant]
    print("Mantisa",mantisa,"con",bits_mant,"bits")

#Vemos el exponente

  #vemos cual es el  maximo que podemos representar con el exponente en decimal (dado los bits del exponente)
  exponente_maximo = int('1'*bits_exp,2) - (2**(bits_exp-1)-1)
  #vemos el exponente que tiene el numero en decimal
  exponente_numero = int(b[1:12],2) - (2**(11-1)-1)

  #print("\nexp max es", exponente_maximo,"con",bits_exp,"bits de exponente")
  #print("exp numero es", exponente_numero,"con 11 bits de exponente")

  #se ve si el numero a representar "cabe" con los bits que nos dan

  #Caso donde no "cabe" el numero con los nuevos bits disponibles
  if exponente_maximo < exponente_numero: 
    
    if b[0]=='1':
      return -np.inf
    else:
      return np.inf
    print("el exponente final es",exponente,"la mantisa es",m,"el resultado es",result,"\n")
  
  
  #caso especial exponente tiene solo numeros "1"
  elif exponente_maximo == exponente_numero:
    
    #caso donde signo positivo y mantisa solo son "0"
    if (m.count('0') == len(m)) and b[0] == '0':
      return np.inf
    
    #caso donde signo es negativo y mantisa solo tiene "0"
    elif (m.count('0') == len(m)) and b[0] == '1':
      return -np.inf

    #caso donde mantisa tiene algun "1"
    elif m.count('1') != 0:
      return None #NaN

  #caso especial exponente tiene solo numeros "0"
  elif int(b[1:12],2) == 0:
    #Numeros sub-normales
    print("si")

  #Caso donde si cabe el numero y no es un caso especial
  else:
    #Convertir numero
    mantisa = '1'+ mantisa

    for iterador in range(0, len(mantisa), 1):   
      if mantisa[iterador] == '1':
        result += 2**(exponente_numero - iterador)
  if b[0] == '1':
    result = result*-1
  
  return  result

f_new_rep(209,2,11)

0 10000000110 1010001000000000000000000000000000000000000000000000
Mantisa 11 con 2 bits


224

In [33]:
"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    result = f_new_rep(n1+n2,bits_mant,bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    result = f_new_rep(n1*n2,bits_mant,bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    result = f_new_rep(n1/n2,bits_mant,bits_exp)
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    return x_roots

In [67]:

# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

bits_mant = 52#PARAMETRO A CAMBIAR
bits_exp = 11 #PARAMETRO A CAMBIAR

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.

print(am,bm,cm)
"""
print("division", f_division(40,45,3,3))
print("suma", f_suma(40,45,3,3))
print("multiplicacion", f_multiplicacion(40,45,3,3))
"""

0 10000010110 0010101110101101011000010011111111101010101000101010
0 10000010000 0010001010000000010000110000000101000010111101100010
0 10000010110 0011000000110111011000100100101111101111101011101000
9819824.624837 148736.523476 9968561.148313


'\nprint("division", f_division(40,45,3,3))\nprint("suma", f_suma(40,45,3,3))\nprint("multiplicacion", f_multiplicacion(40,45,3,3))\n'

### 2. Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
T(t,α,β)=α⋅cos(t)+β⋅sin(log(t)). 

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros  α  y  β  los cuales corresponden a las condiciones iniciales del experimento. Por otro lado  t  representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros  α  y  β  y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


Explicación:

En primer lugar, sabemos para que valor de "x" (o cercano a este) debe haber una raiz, este valor es 25.

En segundo lugar, notamos que la función de por si tiene raices cerca al menos, e incluso a través de su portomaniento en general, se puede determinar que entre un rango de valores, si o si debe haber una raiz, para este caso puntual el rango inicial es [22,27].

En tercer lugar, dado el intervalo previamente definido, se utiliza el metódo de la bisección, ya que ya poseeemos una referncia de donde se encuentran las raices. Este metódo, en caso de encontrarse con dos soluciones de igual signo, procede a llamar recursivamente al metódo, no obstante, se acota el intervalo en 0.5 cada extremo, esto provoca que busque la solución tal que tengan diferente signo, finalemnte retorna el valor.

In [19]:
#Función Auxiliar

#Ref[2]
def bisect(f, a, b, tol, maxNumberIterations=100):
    fa = f(a)
    fb = f(b)
    i = 0
    # Just checking if the sign is not negative => not root  necessarily 
    if np.sign(f(a)*f(b)) >= 0:
        return np.double(bisect(f,a+0.5,b-0.5,tol))
    
    while ((b-a)/2 > tol) and i<=maxNumberIterations:
        c = (a+b)/2.
        fc = f(c)
        # Did we find the root?
        if fc == 0:
            print('f(c)==0')
            break
        elif np.sign(fa*fc) < 0:
            b = c
            fb = fc
        else:
            a = c
            fa = fc
        i += 1
        
    xc = (a+b)/2.
    return np.double(xc)   


In [21]:

"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""
def raiz_de_oreman(alpha, beta, tol = 1e-10): 

    function= lambda x: ((alpha*np.cos(x)+beta*np.sin(np.log(x)))/(x-5))*(x-5)
    
    #intervalo [22,27]
    root = bisect(function,22,27,tol,30)

    return np.double(root)

alpha = 82.0 #PARAMETROS VARIABLES
beta = 200.0 #PARAMETROS VARIABLES

raiz_de_oreman(alpha,beta)

23.61103859168361

# Referencias
[1] Función f_new_rep: Función que utilizar ciertas lineas de los jupyter de los ramos, una de ellas es pasar numero a binario.

[2] Funcion bisect: Función modificada de los jupyter del ramo.